In [28]:
import pandas as pd 
import numpy as np 
import os
import cv2
import matplotlib.pyplot as plt
import warnings
import scipy

import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing import image, image_dataset_from_directory
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications.inception_v3 import InceptionV3 # InceptionV3



print("Num GPUs Available: ", len(tensorflow.config.list_physical_devices('GPU')))


# Set the seed value for experiment reproduci.bility.
seed = 1842
tensorflow.random.set_seed(seed)
np.random.seed(seed)
# Turn off warnings for cleaner looking notebook
warnings.simplefilter('ignore')

Num GPUs Available:  1


In [20]:
#define image dataset 
#why do we rescale?
image_generator = ImageDataGenerator(rescale=1/255, validation_split=0, shear_range =.25, zoom_range =.2, horizontal_flip = True, rotation_range=20)     


train_dataset = image_generator.flow_from_directory(batch_size=32,
                                                 directory='../train',
                                                 shuffle=True,
                                                 target_size=(224,224), 
                                                 subset="training",
                                                 class_mode='binary')

image_generator = ImageDataGenerator(rescale=1/255) 

validation_dataset = image_generator.flow_from_directory(batch_size=32,
                                                 directory='../valid',
                                                 shuffle=True,
                                                 target_size=(224,224),
                                                 class_mode='binary')


test_dataset = image_generator.flow_from_directory(
                                                 directory='../test',
                                                 shuffle=False,
                                                 target_size=(224,224), 
                                                 class_mode=None)

Found 2600 images belonging to 2 classes.
Found 870 images belonging to 2 classes.
Found 864 images belonging to 2 classes.


In [29]:
model_inception = InceptionV3(input_shape=(224,224,3), weights='imagenet', include_top=False)

for layer in model_inception.layers:
    layer.trainable = False
#x = Flatten()(model_inception.output)

tf.random.set_seed(150)

model = tf.keras.models.Sequential([
    model_inception,
    GlobalAveragePooling2D(),    
    Dense(512,activation="relu"),
    Dropout(0.4),
    # Dense(256,activation="relu"),
    # Dropout(0.4),
    Dense(128, activation='swish'),  
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(),
              loss="binary_crossentropy",
              metrics=['accuracy'])

In [30]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Functional)    (None, 5, 5, 2048)        21802784  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense_18 (Dense)             (None, 512)               1049088   
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 128)               65664     
_________________________________________________________________
dense_20 (Dense)             (None, 1)                 129       
Total params: 22,917,665
Trainable params: 1,114,881
Non-trainable params: 21,802,784
__________________________________

In [13]:
prediction = Dense(1, activation='sigmoid')(x)

modelinception = Model(inputs=model_inception.input, outputs=prediction)

modelinception.compile(optimizer='adam',
loss=tensorflow.losses.BinaryCrossentropy(),
metrics=[keras.metrics.AUC(name='auc'),'acc'])

#callback = keras.callbacks.EarlyStopping(monitor='val_loss',
#                                            patience=8,
#                                            restore_best_weights=True

In [31]:
model.fit(train_dataset, epochs=10, validation_data=validation_dataset)

Epoch 1/10


In [ ]:
model.evaluate(test_dataset)

RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.